### Импорт библиотек

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from nltk.tokenize import word_tokenize
import nltk
nltk.download("punkt")

import seaborn
seaborn.set_theme(palette="summer")

import numpy as np
import matplotlib.pyplot as plt
import datasets
from tqdm.auto import tqdm
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from collections import Counter
from typing import List
import string

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

[nltk_data] Downloading package punkt to /home/vitalii/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


cpu


### Загрузка набора данных

In [2]:
dataset = datasets.load_dataset("imdb")
pass

### Подсчёт частоты вхождения слов для составления словаря

In [3]:
words = Counter()

for example in tqdm(dataset["train"]["text"]):
    # Приводим к нижнему регистру и убираем пунктуацию
    processed_text = example.lower().translate(
        str.maketrans('', '', string.punctuation)
    )

    # Находим все слова и считаем их частоту вхождения
    for word in word_tokenize(processed_text):
        words[word] += 1

  0%|          | 0/25000 [00:00<?, ?it/s]

In [4]:
# Установка специальных токенов:
# <unk> - неизвестный токен
# <bos>, <eos> - начало и конец последовательности
# <pad> - объединение последовательностей разных длин в один батч
vocab = set(["<unk>", "<bos>", "<eos>", "<pad>"])
# Добавление слов, которые встречались >25 раз
counter_threshold = 25

for char, cnt in words.items():
    if cnt > counter_threshold:
        vocab.add(char)

len(vocab)

11399

In [5]:
# Словари word2ind и ind2word
# осуществляют mapping из слов в индексы и наоборот
word2ind = {char: i for i, char in enumerate(vocab)}
ind2word = {i: char for char, i in word2ind.items()}

### Работа с torch dataloader
Класс Dataset должен имплеминтировать 3 пункта:
- Конструктор для создания объекта класса
- \_\_getitem__ - получение по индексу объект набора данных
- \_\_len__ - длина набора данных

In [6]:
class WordDataset:
    def __init__(self, sentences):
        self.data = sentences
        self.unk_id = word2ind["<unk>"]
        self.bos_id = word2ind["<bos>"]
        self.eos_id = word2ind["<eos>"]
        self.pad_id = word2ind["<pad>"]

    def __getitem__(self, idx: int) -> List[int]:
        processed_text = self.data[idx]["text"].lower().translate(
            str.maketrans('', '', string.punctuation)
        )
        tokenized_sentence = [self.bos_id]
        tokenized_sentence += [
            word2ind.get(word, self.unk_id) for word in word_tokenize(
                processed_text
            )
        ]
        tokenized_sentence += [self.eos_id]

        train_sample = {
            "text": tokenized_sentence,
            "label": self.data[idx]["label"]
        }

        return train_sample
    
    def __len__(self) -> int:
        return len(self.data)

In [7]:
def collate_fn_with_padding(
    input_batch: List[List[int]], pad_id=word2ind["<pad>"], max_len=256
) -> torch.Tensor:
    seq_lens = [len(x["text"]) for x in input_batch]
    max_seq_len = min(max(seq_lens), max_len)

    new_batch = []
    for sequence in input_batch:
        sequence["text"] = sequence["text"][:max_seq_len]
        for _ in range(max_seq_len - len(sequence["text"])):
            sequence["text"].append(pad_id)

        new_batch.append(sequence["text"])

    sequence = torch.LongTensor(new_batch).to(device)
    labels = torch.LongTensor([x["label"] for x in input_batch]).to(device)

    new_batch = {
        "input_ids": sequence,
        "label": labels
    }

    return new_batch

In [8]:
train_dataset = WordDataset(dataset["train"])

np.random.seed(42)
idx = np.random.choice(np.arange(len(dataset["test"])), 2000)
eval_dataset = WordDataset(dataset["test"].select(idx))

batch_size = 128
train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=collate_fn_with_padding,
    batch_size=batch_size
)

eval_dataloader = DataLoader(
    eval_dataset,
    shuffle=False,
    collate_fn=collate_fn_with_padding,
    batch_size=batch_size
)

### Архитектура модели

In [ ]:
class CharLM(nn.Module):
    def __init__(
        self, hidden_dim: int, vocab_size: int,
        num_classes: int = 2,
        aggregation_type: str = "max"
    ):
    pass
        